In [9]:
import numpy as np
import pandas as pd
import glob
from model.ctabgan import CTABGAN
df = pd.read_csv("Real_Datasets/car.csv")

In [10]:
df = df.drop(columns=['Year','Model'])
df.head(10)

,Brand,Engine_Size,Fuel_Type,Transmission,Mileage,Doors,Owner_Count,Price
0,Kia,4.2,Diesel,Manual,289944,3,5,8501
1,Chevrolet,2.0,Hybrid,Automatic,5356,2,3,12092
2,Mercedes,4.2,Diesel,Automatic,231440,4,2,11171
3,Audi,2.0,Electric,Manual,160971,2,1,11780
4,Volkswagen,2.6,Hybrid,Semi-Automatic,286618,3,3,2867
5,Toyota,2.7,Petrol,Automatic,157889,4,4,7242
6,Honda,3.4,Electric,Automatic,139584,3,1,11208
7,Kia,4.7,Electric,Semi-Automatic,157495,2,2,7950
8,Kia,2.6,Hybrid,Manual,98700,3,4,9926
9,Toyota,3.1,Petrol,Manual,107724,2,5,6545


In [7]:
df["Model"].unique().shape[0]

30

In [3]:
synthesizer =  CTABGAN(df,
                 test_ratio = 0.20,
                 categorical_columns = ["Brand","Model","Fuel_Type","Transmission"], 
                 log_columns = [],
                 mixed_columns= {},
                 general_columns = [],
                 non_categorical_columns = [],
                 integer_columns = [],
                 problem_type= {"Classification": 'Price'}) 

In [6]:
synthesizer.fit(1)

IndexError: arrays used as indices must be of integer (or boolean) type